In [2]:
import pyspark

In [3]:
    sc = pyspark.SparkContext('local[*]')
    # Example usage:
    rdd = sc.parallelize(range(1000))
    print(rdd.takeSample(False, 5))

[788, 360, 863, 55, 89]


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TestSpark") \
    .getOrCreate()

spark


In [5]:
df = spark.createDataFrame([(1, "Hola"), (2, "Spark"), (3, "Docker")], ["id", "mensaje"])
df.show()

print("Total filas:", df.count())


+---+-------+
| id|mensaje|
+---+-------+
|  1|   Hola|
|  2|  Spark|
|  3| Docker|
+---+-------+

Total filas: 3


In [7]:
!mkdir -p /home/jovyan/jars

In [8]:
!wget -P /home/jovyan/jars https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.13.30/snowflake-jdbc-3.13.30.jar

!wget -P /home/jovyan/jars https://repo1.maven.org/maven2/net/snowflake/spark-snowflake_2.12/2.12.0-spark_3.4/spark-snowflake_2.12-2.12.0-spark_3.4.jar

--2025-10-16 14:30:45--  https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.13.30/snowflake-jdbc-3.13.30.jar
Resolving repo1.maven.org (repo1.maven.org)... 104.18.18.12, 104.18.19.12, 2606:4700::6812:120c, ...
Connecting to repo1.maven.org (repo1.maven.org)|104.18.18.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33419306 (32M) [application/java-archive]
Saving to: ‘/home/jovyan/jars/snowflake-jdbc-3.13.30.jar’

snowflake-jdbc-3.13 100%[===================>]  31.87M  8.22MB/s    in 3.9s    

2025-10-16 14:30:50 (8.12 MB/s) - ‘/home/jovyan/jars/snowflake-jdbc-3.13.30.jar’ saved [33419306/33419306]

--2025-10-16 14:30:50--  https://repo1.maven.org/maven2/net/snowflake/spark-snowflake_2.12/2.12.0-spark_3.4/spark-snowflake_2.12-2.12.0-spark_3.4.jar
Resolving repo1.maven.org (repo1.maven.org)... 104.18.18.12, 104.18.19.12, 2606:4700::6812:120c, ...
Connecting to repo1.maven.org (repo1.maven.org)|104.18.18.12|:443... connected.
HTTP request sent, awai

In [9]:
!ls /home/jovyan/jars

snowflake-jdbc-3.13.30.jar  spark-snowflake_2.12-2.12.0-spark_3.4.jar


In [13]:
spark.stop()

In [15]:

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SnowflakeTest") \
    .config("spark.jars", "/home/jovyan/jars/snowflake-jdbc-3.13.30.jar,/home/jovyan/jars/spark-snowflake_2.12-2.12.0-spark_3.4.jar") \
    .getOrCreate()

In [16]:
import os

sfOptions = {
    "sfURL": f"https://{os.getenv('SF_ACCOUNT')}.snowflakecomputing.com",
    "sfUser": os.getenv("SF_USER"),
    "sfPassword": os.getenv("SF_PASSWORD"),
    "sfWarehouse": os.getenv("SF_WAREHOUSE"),
    "sfDatabase": os.getenv("SF_DATABASE"),
    "sfSchema": os.getenv("SF_SCHEMA"),
    "sfRole": os.getenv("SF_ROLE")
}

df = spark.read \
    .format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "INFORMATION_SCHEMA.TABLES") \
    .load()

df.show(5)


Py4JJavaError: An error occurred while calling o124.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: snowflake. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:172)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	a